## Orchestrator-Workers Workflow
이 워크플로우에서는 중앙 LLM이 동적으로 작업을 분해하고, 이를 worker LLM들에게 위임한 뒤, 그들의 결과를 종합합니다.

### 이 워크플로우의 적용 시점
이 워크플로우는 필요한 하위 작업을 미리 예측할 수 없는 복잡한 작업에 적합합니다. 단순 병렬화와의 주요 차이점은 유연성에 있습니다 - 하위 작업들이 미리 정의되지 않고, 특정 입력에 기반하여 orchestrator가 결정합니다.

In [1]:
from typing import Dict, List, Optional
from util import llm_call, extract_xml

def parse_tasks(tasks_xml: str) -> List[Dict]:
    """XML 작업을 작업 사전 목록으로 파싱하세요."""
    tasks = []
    current_task = {}
    
    for line in tasks_xml.split('\n'):
        line = line.strip()
        if not line:
            continue
            
        if line.startswith("<task>"):
            current_task = {}
        elif line.startswith("<type>"):
            current_task["type"] = line[6:-7].strip()
        elif line.startswith("<description>"):
            current_task["description"] = line[12:-13].strip()
        elif line.startswith("</task>"):
            if "description" in current_task:
                if "type" not in current_task:
                    current_task["type"] = "default"
                tasks.append(current_task)
    
    return tasks

class FlexibleOrchestrator:
    """worker LLM을 사용하여 작업을 세분화하고 병렬로 실행하세요."""
    
    def __init__(
        self,
        orchestrator_prompt: str,
        worker_prompt: str,
    ):
        """프롬프트 템플릿으로 초기화합니다."""
        self.orchestrator_prompt = orchestrator_prompt
        self.worker_prompt = worker_prompt

    def _format_prompt(self, template: str, **kwargs) -> str:
        """변수로 프롬프트 템플릿의 형식을 지정합니다."""
        try:
            return template.format(**kwargs)
        except KeyError as e:
            raise ValueError(f"Missing required prompt variable: {e}")

    def process(self, task: str, context: Optional[Dict] = None) -> Dict:
        """작업을 세분화하고 하위 작업을 병렬로 실행하여 작업을 처리합니다."""
        context = context or {}
        
        # Step 1: Get orchestrator response
        orchestrator_input = self._format_prompt(
            self.orchestrator_prompt,
            task=task,
            **context
        )
        orchestrator_response = llm_call(orchestrator_input)
        
        # Parse orchestrator response
        analysis = extract_xml(orchestrator_response, "analysis")
        tasks_xml = extract_xml(orchestrator_response, "tasks")
        tasks = parse_tasks(tasks_xml)
        
        print("\n=== ORCHESTRATOR OUTPUT ===")
        print(f"\nANALYSIS:\n{analysis}")
        print(f"\nTASKS:\n{tasks}")
        
        # Step 2: Process each task
        worker_results = []
        for task_info in tasks:
            worker_input = self._format_prompt(
                self.worker_prompt,
                original_task=task,
                task_type=task_info['type'],
                task_description=task_info['description'],
                **context
            )
            
            worker_response = llm_call(worker_input)
            result = extract_xml(worker_response, "response")
            
            worker_results.append({
                "type": task_info["type"],
                "description": task_info["description"],
                "result": result
            })
            
            print(f"\n=== WORKER RESULT ({task_info['type']}) ===\n{result}\n")
        
        return {
            "analysis": analysis,
            "worker_results": worker_results,
        }


### Example Use Case: Marketing Variation Generation



In [3]:
ORCHESTRATOR_PROMPT = """
이 과제를 분석하고 2-3가지 서로 다른 접근 방식으로 나누세요:

Task: {task}

답변을 다음 형식으로 제출하세요:

<analysis>
과제에 대한 이해와 어떤 변형이 가치 있을지 설명하세요.
각 접근 방식이 과제의 다른 측면들을 어떻게 다루는지에 초점을 맞추세요. 
</analysis>

<tasks>
    <task>
    <type>formal</type>
    <description>명세에 중점을 둔 정확하고 기술적인 버전 작성</description>
    </task>
    <task>
    <type>conversational</type>
    <description>독자와 공감대를 형성하는 친근하고 매력적인 버전 작성</description>
    </task>
</tasks>
"""

WORKER_PROMPT = """
다음을 기반으로 콘텐츠를 생성하세요:
Task: {original_task}
Style: {task_type}
Guidelines: {task_description}

답변을 다음 형식으로 제출하세요:

<response>
지정된 스타일을 유지하고 요구사항을 완전히 충족하는 귀하의 콘텐츠를 여기에 작성하세요.
</response>
"""


orchestrator = FlexibleOrchestrator(
    orchestrator_prompt=ORCHESTRATOR_PROMPT,
    worker_prompt=WORKER_PROMPT,
)

results = orchestrator.process(
    task="친환경 물병에 대한 제품 설명 작성",
    context={
        "target_audience": "환경 의식이 있는 밀레니얼 세대",
        "key_features": ["플라스틱 프리", "단열", "평생 보증"]
    }
)


=== ORCHESTRATOR OUTPUT ===

ANALYSIS:

친환경 물병 제품 설명은 기술적 특성과 환경적 가치를 모두 전달해야 하는 복합적인 과제입니다. 주요 고려사항:
- 제품의 물리적/기술적 특성
- 환경적 이점과 지속가능성
- 소비자의 실용적/감성적 니즈
- 구매 동기 부여

이를 위해 다음과 같이 서로 다른 접근방식으로 나눌 수 있습니다.


TASKS:
[{'type': 'technical-environmental', 'description': ''}, {'type': 'lifestyle-emotional', 'description': ''}, {'type': 'practical-benefit', 'description': ''}]

=== WORKER RESULT (technical-environmental) ===

친환경 재사용 물병 - 에코라이프 시리즈

제품 사양:
• 소재: 식품등급 18/8 스테인리스스틸 (BPA-free)
• 용량: 600ml
• 중량: 280g
• 크기: 높이 24cm x 지름 7cm
• 이중 진공 단열 구조

환경적 특징:
- 100% 재활용 가능한 소재 사용
- 일회용 플라스틱 물병 대비 연간 최대 1,460개 절감 효과
- 제조 과정에서 탄소 배출량 최소화 공정 적용
- 포장재는 생분해성 재료 사용

기술적 특성:
• 24시간 보냉/12시간 보온 성능
• 누수 방지 트리플 실링 시스템
• 항균 처리된 병목 설계
• 내구성 강화 특수 코팅 처리

지속가능성 인증:
- ISO 14001 환경경영시스템 인증
- 탄소발자국 인증
- 친환경 제조공정 인증

사용자 및 환경을 모두 고려한 혁신적 디자인으로, 일상적인 수분 섭취와 동시에 지구 환경 보호에 기여합니다.



=== WORKER RESULT (lifestyle-emotional) ===

당신의 일상에 작은 변화로 시작하는 지구를 위한 여정

매일 아침, 투명한 물방울이 맺히는 순간부터 시작되는 특별한 경험. 
자연에서 영감을 받아 디자인된 우리의 